In [34]:
import collections
from urllib.request import urlopen, Request
import json
import pandas as pd
import datetime
# import pytz

In [20]:
locations = {

    "22.258351741017634, 114.21163275365531" : "Tai Tam Reservoir" ,
    "22.27054210988102, 114.00032030347596"  : "Mui Wo",
    "22.540797536909047, 114.43437970480949" : "Tung Ping Chau",
    "22.29770991812577, 114.30310328185212" : "Clear Water Bay",
    "22.239390991398583, 114.24173429924463" : "Dragon's Back",
    "22.23398579735171, 114.15930196830752" : "Ap Lei Chau",
    "22.28617189441449, 114.04065619628331" : "Peng Chau"

}

locations = collections.OrderedDict(sorted(locations.items()))

In [32]:
loc_name_gps = {} # reverse lookup
for k, v in locations.items():
    loc_name_gps[v] = k

In [25]:
def get_weather_df(latlong):
  # Construct URL
  _gps_loc = latlong.split(",")
  latitude = str(round(float(_gps_loc[0].strip()),4))
  longitude = str(round(float(_gps_loc[1].strip()),4))

  # API call
  # Params. Ref : https://open-meteo.com/en/docs/#latitude=22.2787&longitude=114.1815&hourly=temperature_2m
  _params = 'precipitation,cloudcover,windgusts_10m,temperature_2m'
  url = 'https://api.open-meteo.com/v1/forecast?latitude=' + latitude + '&longitude=' \
    + longitude +'&hourly={}&timezone=auto'.format(_params)

  # Retrieve data
  request = Request(url) # <class 'urllib.request.Request'>
  response = urlopen(request) # <class 'http.client.HTTPResponse'>
  forecast = response.read() # <class 'bytes'>
  data = json.loads(forecast) # <class 'dic t'>
  df_json = pd.json_normalize(data) # <class 'pandas.core.frame.DataFrame'>
  
  return df_json
  

# Get Forecast - Single Location

In [28]:
df1 = get_weather_df(loc_name_gps["Peng Chau"])

# # df1[column][0]
# for e in df1.columns:
#     try:
#         if e.startswith("hourly."):
#             print(f"{e:<30}\tlen={len(df1[e][0]):<30}")        
#     except TypeError:
#         pass

# Dataframe index is time, columns are weather parameters
df2 = pd.DataFrame()
df2["time"] = df1["hourly.time"][0]
df2["precipitation"] = df1["hourly.precipitation"][0]
df2["cloudcover"] = df1["hourly.cloudcover"][0]
df2["windgusts_10m"] = df1["hourly.windgusts_10m"][0]
df2["temperature_2m"] = df1["hourly.temperature_2m"][0]

print("Before applying filter")
df2.head()

Before applying filter


,time,precipitation,cloudcover,windgusts_10m,temperature_2m
0,2024-01-25T00:00,0.0,72,22.7,8.6
1,2024-01-25T01:00,0.0,67,22.7,8.4
2,2024-01-25T02:00,0.0,86,22.3,8.6
3,2024-01-25T03:00,0.0,95,23.0,8.6
4,2024-01-25T04:00,0.0,95,22.7,8.7


In [31]:
# Filers:
MAX_PRECIP_mm = 0
MAX_CLOUDS_pcnt = 100
MAX_WIND_kph = 35

# Masks for filtering
mask_precip = df2["precipitation"] <= MAX_PRECIP_mm
mask_clouds = df2["cloudcover"] <= MAX_CLOUDS_pcnt
mask_wind = df2["windgusts_10m"] <= MAX_WIND_kph

df3 = df2[mask_precip & mask_clouds & mask_wind]

# Insert column for day of week using  datetime.date(yyyy,mm,dd).strftime('%A')
list_daysofweek = []
for str_datetime in df3['time']:
    substr1   = str_datetime.split('-')
    yyyy, mm  = int(substr1[0]), int(substr1[1])
    substr2     = substr1[2].split('T')
    dd, hh      = int(substr2[0]), int(substr2[1].split(':')[0])
    list_daysofweek.append(datetime.date(yyyy,mm,dd).strftime('%A'))

df3.insert(0, "dayofweek", list_daysofweek, True)
df3.head()

,dayofweek,time,precipitation,cloudcover,windgusts_10m,temperature_2m
0,Thursday,2024-01-25T00:00,0.0,72,22.7,8.6
1,Thursday,2024-01-25T01:00,0.0,67,22.7,8.4
2,Thursday,2024-01-25T02:00,0.0,86,22.3,8.6
3,Thursday,2024-01-25T03:00,0.0,95,23.0,8.6
4,Thursday,2024-01-25T04:00,0.0,95,22.7,8.7


# Get Forecast - Multi. Location

In [35]:
# TODO